In [ ]:
#pip install psycopg2-binary

In [ ]:
import sqlalchemy
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, inspect

In [28]:
df_raw = pd.read_csv('data/Airline_review.csv')
df_raw

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23166,23166,ZIPAIR,1,"""customer service is terrible""",5th July 2022,False,Bangkok to Tokyo. I’ve flown many low cost ai...,NaN,Couple Leisure,Economy Class,Bangkok to Tokyo,June 2022,2.0,1.0,NaN,1.0,1.0,1.0,1.0,no
23167,23167,ZIPAIR,1,"""Avoid at all costs""",1st June 2022,True,Avoid at all costs. I booked flights to go f...,NaN,Solo Leisure,Economy Class,Singapore to Tokyo,June 2022,NaN,NaN,NaN,NaN,NaN,NaN,1.0,no
23168,23168,ZIPAIR,3,"""Will not recommend to anyone""",31st May 2022,True,Flight was leaving at 23.15 and after an hou...,NaN,Business,Economy Class,Bangkok to Tokyo,May 2022,2.0,4.0,2.0,1.0,NaN,NaN,2.0,no
23169,23169,ZIPAIR,6,"""It was immaculately clean""",23rd May 2022,True,Zipair is JAL’s budget airline. They don’t ha...,Dreamliner,Business,Business Class,Tokyo to Los Angeles,May 2022,3.0,4.0,3.0,1.0,2.0,5.0,5.0,yes


In [30]:
needed_columns = [c for c in df_raw.columns if c != 'Unnamed: 0']
needed_columns

['Airline Name',
 'Overall_Rating',
 'Review_Title',
 'Review Date',
 'Verified',
 'Review',
 'Aircraft',
 'Type Of Traveller',
 'Seat Type',
 'Route',
 'Date Flown',
 'Seat Comfort',
 'Cabin Staff Service',
 'Food & Beverages',
 'Ground Service',
 'Inflight Entertainment',
 'Wifi & Connectivity',
 'Value For Money',
 'Recommended']

In [43]:
df = df_raw[needed_columns]
df['Review_Date'] = pd.to_datetime(df['Review Date'], format="mixed")
df['Flight_Date'] = pd.to_datetime(df['Date Flown'], format="mixed")
df

,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,...,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,Review_Date,Flight_Date
0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,...,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes,2019-11-11,2019-11-01
1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,...,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no,2019-06-25,2019-06-01
2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,...,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no,2019-06-25,2019-06-01
3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no,2019-09-28,2019-09-01
4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no,2019-09-24,2019-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23166,ZIPAIR,1,"""customer service is terrible""",5th July 2022,False,Bangkok to Tokyo. I’ve flown many low cost ai...,NaN,Couple Leisure,Economy Class,Bangkok to Tokyo,...,2.0,1.0,NaN,1.0,1.0,1.0,1.0,no,2022-07-05,2022-06-01
23167,ZIPAIR,1,"""Avoid at all costs""",1st June 2022,True,Avoid at all costs. I booked flights to go f...,NaN,Solo Leisure,Economy Class,Singapore to Tokyo,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,no,2022-06-01,2022-06-01
23168,ZIPAIR,3,"""Will not recommend to anyone""",31st May 2022,True,Flight was leaving at 23.15 and after an hou...,NaN,Business,Economy Class,Bangkok to Tokyo,...,2.0,4.0,2.0,1.0,NaN,NaN,2.0,no,2022-05-31,2022-05-01
23169,ZIPAIR,6,"""It was immaculately clean""",23rd May 2022,True,Zipair is JAL’s budget airline. They don’t ha...,Dreamliner,Business,Business Class,Tokyo to Los Angeles,...,3.0,4.0,3.0,1.0,2.0,5.0,5.0,yes,2022-05-23,2022-05-01


In [45]:
#Connect to RDS
conf ={
    'host':"data608project-postgresql.cti8i5zrv6mp.us-east-1.rds.amazonaws.com",
    'port':'5432',
    'database':"data608project-postgresql",
    'user':"postgres",
    'password':"Data608projectbigdata"
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

df.to_sql('bronze.kaggle_airline_reviews', engine, index=False, if_exists='replace')

171

In [46]:
#if encounter error with psycopg2: pip install psycopg2-binary
import sqlalchemy
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, inspect


#Connect to RDS
conf ={
    'host':"data608project-postgresql.cti8i5zrv6mp.us-east-1.rds.amazonaws.com",
    'port':'5432',
    'database':"data608project-postgresql",
    'user':"postgres",
    'password':"Data608projectbigdata"
}

engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

#inspect whats on the RDS
inspector = inspect(engine)
tables = inspector.get_table_names()
print(tables)

['bronze.kaggle_airline_reviews']


In [47]:
#if encounter error with psycopg2: pip install psycopg2-binary
import sqlalchemy
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, inspect


#Connect to RDS
conf ={
    'host':"data608project-postgresql.cti8i5zrv6mp.us-east-1.rds.amazonaws.com",
    'port':'5432',
    'database':"data608project-postgresql",
    'user':"postgres",
    'password':"Data608projectbigdata"
}

engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

#take a look at the table
sql="""
SELECT COUNT(*) FROM "bronze.kaggle_airline_reviews"
"""
df_back = pd.read_sql_query(sql, engine)
print(df_back)

   count
0  23171


#### Upload Dataframe to Databricks volume

In [8]:
import requests

In [4]:
%run POC_DatabricksToken.ipynb

In [16]:
#Using Databricks Upload a file API
#https://docs.databricks.com/api/workspace/files/upload
databricks_instance = 'dbc-daf0509f-1d8b.cloud.databricks.com'
local_path = 'data/Airline_review.csv'
remote_path = '/Volumes/workspace/data608/data608_volume/Kaggle_airline_reviews.csv'
headers = {'Authorization' : f"Bearer {DATABRICKS_TOKEN}"}
#print(headers)

with open(local_path, 'rb') as f:
    r = requests.put(
      f"https://{databricks_instance}/api/2.0/fs/files{remote_path}?overwrite=true",
      headers=headers,
      data=f)
    r.raise_for_status()

print(f"Uploaded '{local_path}' to '{remote_path}' on Databricks workspace {databricks_instance}")

Uploaded 'data/Airline_review.csv' to '/Volumes/workspace/data608/data608_volume/Kaggle_airline_reviews.csv' on Databricks workspace dbc-daf0509f-1d8b.cloud.databricks.com
